RNA het pipe

pull all red clover assemblies and cat into reference

In [2]:
%%bash
wget ftp://ftp.ensemblgenomes.org/pub/plants/release-38/fasta/trifolium_pratense/dna/Trifolium_pratense.Trpr.dna.chromosome.LG1.fa.gz
wget ftp://ftp.ensemblgenomes.org/pub/plants/release-38/fasta/trifolium_pratense/dna/Trifolium_pratense.Trpr.dna.chromosome.LG2.fa.gz
wget ftp://ftp.ensemblgenomes.org/pub/plants/release-38/fasta/trifolium_pratense/dna/Trifolium_pratense.Trpr.dna.chromosome.LG3.fa.gz
wget ftp://ftp.ensemblgenomes.org/pub/plants/release-38/fasta/trifolium_pratense/dna/Trifolium_pratense.Trpr.dna.chromosome.LG4.fa.gz
wget ftp://ftp.ensemblgenomes.org/pub/plants/release-38/fasta/trifolium_pratense/dna/Trifolium_pratense.Trpr.dna.chromosome.LG5.fa.gz
wget ftp://ftp.ensemblgenomes.org/pub/plants/release-38/fasta/trifolium_pratense/dna/Trifolium_pratense.Trpr.dna.chromosome.LG6.fa.gz
wget ftp://ftp.ensemblgenomes.org/pub/plants/release-38/fasta/trifolium_pratense/dna/Trifolium_pratense.Trpr.dna.chromosome.LG7.fa.gz
gunzip *LG*.fa.gz
cat *LG*.fa > redclover_ref.fa
#gff:
wget ftp://ftp.ensemblgenomes.org/pub/plants/release-38/gff3/trifolium_pratense/Trifolium_pratense.Trpr.38.chr.gff3.gz

--2018-05-09 16:17:44--  ftp://ftp.ensemblgenomes.org/pub/plants/release-38/fasta/trifolium_pratense/dna/Trifolium_pratense.Trpr.dna.chromosome.LG1.fa.gz
           => ‘Trifolium_pratense.Trpr.dna.chromosome.LG1.fa.gz’
Resolving ftp.ensemblgenomes.org (ftp.ensemblgenomes.org)... 193.62.197.94
Connecting to ftp.ensemblgenomes.org (ftp.ensemblgenomes.org)|193.62.197.94|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/plants/release-38/fasta/trifolium_pratense/dna ... done.
==> SIZE Trifolium_pratense.Trpr.dna.chromosome.LG1.fa.gz ... 7617692
==> PASV ... done.    ==> RETR Trifolium_pratense.Trpr.dna.chromosome.LG1.fa.gz ... done.
Length: 7617692 (7.3M) (unauthoritative)

     0K .......... .......... .......... .......... ..........  0%  155K 48s
    50K .......... .......... .......... .......... ..........  1%  307K 36s
   100K .......... .......... .......... .......... ..........  2%  298K 32s
   1

test run with one sample

In [ ]:
%%bash
docker run --rm -v "$(pwd)":/data -w /data inutano/sra-toolkit fastq-dump ERR1665297 


In [22]:
%%bash
fastqp ERR1665297.fastq

ERR1665297.fastq	reads	None	20856498
ERR1665297.fastq	read_len	51	1604346
ERR1665297.fastq	q05	1	26
ERR1665297.fastq	q25	1	30
ERR1665297.fastq	q50	1	31
ERR1665297.fastq	q75	1	31
ERR1665297.fastq	q95	1	34
ERR1665297.fastq	q05	2	28
ERR1665297.fastq	q25	2	31
ERR1665297.fastq	q50	2	31
ERR1665297.fastq	q75	2	33
ERR1665297.fastq	q95	2	34
ERR1665297.fastq	q05	3	26
ERR1665297.fastq	q25	3	31
ERR1665297.fastq	q50	3	31
ERR1665297.fastq	q75	3	34
ERR1665297.fastq	q95	3	34
ERR1665297.fastq	q05	4	28
ERR1665297.fastq	q25	4	35
ERR1665297.fastq	q50	4	35
ERR1665297.fastq	q75	4	37
ERR1665297.fastq	q95	4	37
ERR1665297.fastq	q05	5	26
ERR1665297.fastq	q25	5	35
ERR1665297.fastq	q50	5	35
ERR1665297.fastq	q75	5	37
ERR1665297.fastq	q95	5	37
ERR1665297.fastq	q05	6	32
ERR1665297.fastq	q25	6	35
ERR1665297.fastq	q50	6	35
ERR1665297.fastq	q75	6	37
ERR1665297.fastq	q95	6	37
ERR1665297.fastq	q05	7	28
ERR1665297.fastq	q25	7	35
ERR1665297.fastq	q50	7	35
ERR1665297.fastq	q75	7	37
ERR1665297.fastq	q95	7	37
ERR1665297.fastq

At 188 bytes per read of 51 length we estimate 27,459,402 reads in input file.
Bin size (-s) set to 13.
Approximately 10% complete at read 2,745,951 in 00:00:14
Approximately 20% complete at read 5,491,889 in 00:00:29
Approximately 30% complete at read 8,237,827 in 00:00:44
Approximately 40% complete at read 10,983,765 in 00:00:59
Approximately 50% complete at read 13,729,703 in 00:01:13
Approximately 60% complete at read 16,475,654 in 00:01:28
Approximately 70% complete at read 19,221,592 in 00:01:43
/home/cvisger/anaconda3/lib/python3.6/site-packages/matplotlib/artist.py:896: MatplotlibDeprecationWarning: The set_axis_bgcolor function was deprecated in version 2.0. Use set_facecolor instead.
  return func(v)
There were 20,856,498 reads in the file. Analysis finished in 00:02:10.


In [20]:
%%bash
docker run --rm -v "$(pwd)":/data -w /data fjukstad/trimmomatic  SE ERR1665297.fastq ERR1665297.trimmed.fq SLIDINGWINDOW:4:30
        

TrimmomaticSE: Started with arguments:
 ERR1665297.fastq ERR1665297.trimmed.fq SLIDINGWINDOW:4:30
Automatically using 1 threads
Quality encoding detected as phred33
Input Reads: 20856487 Surviving: 17829749 (85.49%) Dropped: 3026738 (14.51%)
TrimmomaticSE: Completed successfully


In [ ]:
%%bash
fastqp ERR1665297.trimmed.fq

In [27]:
%%bash
docker run --rm -v "$(pwd)":/data -w /data biocontainers/bwa bwa index chrom0.5.fa

[bwa_index] Pack FASTA... 0.13 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 6.63 seconds elapse.
[bwa_index] Update BWT... 0.09 sec
[bwa_index] Pack forward-only FASTA... 0.08 sec
[bwa_index] Construct SA from BWT and Occ... 2.14 sec
[main] Version: 0.7.15-r1140
[main] CMD: bwa index chrom0.5.fa
[main] Real time: 9.745 sec; CPU: 9.117 sec


Make an extreme example dataset

In [ ]:
touch all.fastq
declare -a arr=("ERR1665298"
"ERR1665299"
"ERR1665300"
"ERR1665301"
"ERR1665302"
"ERR1665303"
"ERR1665304"
"ERR1665305"
"ERR1665306"
"ERR1665307"
"ERR1665308")

for i in "${arr[@]}"
do
	docker run --rm -v "$(pwd)":/data -w /data inutano/sra-toolkit fastq-dump "$i"
	cat "$i".fastq >> all.fastq
    rm -f "$i".fastq
done

In [24]:
%%bash
cat Trifolium_pratense.Trpr.dna.chromosome.LG1.fa | wc -l

469035


In [25]:
%%bash
head -n 234500 Trifolium_pratense.Trpr.dna.chromosome.LG1.fa > chrom0.5.fa


In [ ]:
%%bash
docker run --rm -v "$(pwd)":/data -w /data biocontainers/bwa bwa index chrom0.5.fa

In [30]:
%%bash
docker run --rm -v "$(pwd)":/data -w /data biocontainers/bwa bwa mem chrom0.5.fa ERR1665297.trimmed.fq > temp

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 234386 sequences (10000018 bp)...
[M::process] read 232776 sequences (10000046 bp)...
[M::mem_process_seqs] Processed 234386 reads in 5.309 CPU sec, 5.177 real sec
[M::process] read 234036 sequences (10000024 bp)...
[M::mem_process_seqs] Processed 232776 reads in 5.804 CPU sec, 5.536 real sec
[M::process] read 234414 sequences (10000042 bp)...
[M::mem_process_seqs] Processed 234036 reads in 5.464 CPU sec, 5.176 real sec
[M::process] read 235636 sequences (10000018 bp)...
[M::mem_process_seqs] Processed 234414 reads in 5.578 CPU sec, 5.318 real sec
[M::process] read 236980 sequences (10000082 bp)...
[M::mem_process_seqs] Processed 235636 reads in 5.479 CPU sec, 5.210 real sec
[M::process] read 233192 sequences (10000013 bp)...
[M::mem_process_seqs] Processed 236980 reads in 5.973 CPU sec, 5.740 real sec
[M::process] read 239890 sequences (10000068 bp)...
[M::mem_process_seqs] Processed 233192 reads in 5.927 CPU sec, 5.647 

In [31]:
%%bash
docker run --rm -v "$(pwd)":/data -w /data biocontainers/samtools samtools view -bS temp > temp1

In [32]:
%%bash
docker run --rm -v "$(pwd)":/data -w /data biocontainers/samtools samtools view -b -F 4 temp 1 > mappedchrom0.5.bam

[main_samview] random alignment retrieval only works for indexed BAM or CRAM files.
